In [1]:
import arviz as az
import io
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import pymc as pm
import pytensor.tensor as pt
import requests
import statsmodels.api as sm
import warnings
import xarray as xr
from pathlib import Path
import urllib.request

In [2]:
countries_iso = ['BEL', 'CHE', 'DNK', 'ESP', 'FIN', 'FRA', 'GBR', 'IRL', 'ITA', 'NLD', 'NOR', 'NOR', 'PRT', 'SWE']
countries_names = ['Belgium', 'Switzerland', 'Denmark', 'Spain', 'Finland', 'France', 'United Kingdom', 'Ireland', 'Italy', 'Netherlands', 'Norway', 'Norway', 'Portugal', 'Sweden']

name_to_iso = dict(zip(countries_names, countries_iso))

Jordà-Schularick-Taylor Macrohistory Database:
- https://www.macrohistory.net/app/download/9834512569/JSTdatasetR6.xlsx?t=1763503850

Long-Term Productivity Database:
- https://www.longtermproductivity.com/download.html

SIPRI Military Expenditure Database:
- https://www.sipri.org/databases/milex

In [3]:
data_path0 = Path.cwd().parent / "data raw" / "JSTdatasetR6.xlsx"
print("Macro dataset")
print("Full file path: ",data_path0)
print("File exists: ",data_path0.exists())
print()

data_path1 = Path.cwd().parent / "data raw" / "BCLDatabase_online_v2.6.xlsx"
print("TFP dataset")
print("Full file path: ",data_path1)
print("File exists: ",data_path1.exists())
print()

data_path2 = Path.cwd().parent / "data raw" / "SIPRI-Milex-data-1949-2024_2.xlsx"
print("Defense spending dataset")
print("Full file path: ",data_path2)
print("File exists: ",data_path2.exists())

Macro dataset
Full file path:  /Users/awalters/escp_phd/govt_spending/data raw/JSTdatasetR6.xlsx
File exists:  True

TFP dataset
Full file path:  /Users/awalters/escp_phd/govt_spending/data raw/BCLDatabase_online_v2.6.xlsx
File exists:  True

Defense spending dataset
Full file path:  /Users/awalters/escp_phd/govt_spending/data raw/SIPRI-Milex-data-1949-2024_2.xlsx
File exists:  True


In [4]:
df_macro = pd.read_excel(data_path0)

In [5]:
#Real in constant 1990 USD (use 1990 exchange rate)
df_macro["gdp_real_lcu_1990"] = df_macro["gdp"] * 100 / df_macro["cpi"]
df_macro["revenue_real_lcu_1990"] = df_macro["revenue"] * 100 / df_macro["cpi"]
df_macro["expenditure_real_lcu_1990"] = df_macro["expenditure"] * 100 / df_macro["cpi"]

xrusd_1990 = (
    df_macro.loc[df_macro["year"].eq(1990)]
      .groupby("iso", as_index=False)["xrusd"]
      .mean()
      .rename(columns={"xrusd": "xrusd_1990"})
)

df_macro = df_macro.merge(xrusd_1990, on="iso", how="left")

df_macro["gdp_real_usd_1990"] = df_macro["gdp_real_lcu_1990"] / df_macro["xrusd_1990"]
df_macro["revenue_real_usd_1990"] = df_macro["revenue_real_lcu_1990"] / df_macro["xrusd_1990"]
df_macro["expenditure_real_usd_1990"] = df_macro["expenditure_real_lcu_1990"] / df_macro["xrusd_1990"]

In [6]:
df_macro["rgdp_pc"] = df_macro["gdp_real_usd_1990"]/df_macro["pop"] #Real GDP per capita
df_macro["def_gdp"] = 100*(df_macro["revenue_real_usd_1990"] -
                      df_macro["expenditure_real_usd_1990"])/df_macro["gdp_real_usd_1990"] #deficit as percent of gdp
df_macro["gov_pc"] = df_macro["expenditure_real_usd_1990"]/df_macro["pop"] #real government spending per capita
df_macro["bill_rate"] = df_macro["bill_rate"]*100 #convert bill rate to %
df_macro["debtgdp"] = df_macro["debtgdp"]*100 #convert debt to %
df_macro["nx_gdp"] = 100*(df_macro["exports"]-df_macro["imports"])/df_macro["gdp"]

In [7]:
df_macro = df_macro[["iso", #country code
                      "year", #year
                      "rgdp_pc", #Real GDP per capita
                      "gov_pc", #Real government spending per capita
                      "bill_rate", #Fed funds rate
                      "def_gdp", #deficit as percent of gdp
                      "debtgdp", #debt as percent of gdp
                      "nx_gdp" #trade balance as percent of GDP
                     ]]

In [8]:
df_tfp = pd.read_excel(data_path1, sheet_name=3)

df_tfp = df_tfp.rename(columns={df_tfp.columns[0]: "year"}).copy()
df_tfp = df_tfp.iloc[:, :-2]
df_tfp = df_tfp.melt(id_vars="year", var_name="iso", value_name="tfp")

In [9]:
df_mil = pd.read_excel(data_path2, sheet_name=6, skiprows=5, na_values=["..."])
df_mil = df_mil.rename(columns={df_mil.columns[0]: "iso"}).drop(columns=df_mil.columns[1])

df_mil = df_mil.melt(id_vars="iso", var_name="year", value_name="mil_gdp")
df_mil["iso"] = df_mil["iso"].replace(name_to_iso)
df_mil["mil_gdp"] = pd.to_numeric(df_mil["mil_gdp"], errors="coerce")
df_mil["mil_gdp"] = df_mil["mil_gdp"]*100

In [10]:
df = df_macro.merge(
    df_tfp,
    on=["year", "iso"],
    how="left"
)

df = df.merge(
    df_mil,
    on=["year", "iso"],
    how="left"
)

df["iso"] = df["iso"].astype("string")

df = df.loc[df["iso"].isin(countries_iso)].copy()

In [11]:
# sanity: drop rows missing iso/year
df = df.dropna(subset=["iso", "year"])

# ensure unique iso-year (if duplicates exist, decide how to aggregate)
dups = df.duplicated(["iso", "year"])
if dups.any():
    df = (df.groupby(["iso", "year"], as_index=False)
            .mean(numeric_only=True))  # or pick .first(), etc.

In [12]:
out = df.loc[(df["year"] >= 1950) & df.isna().any(axis=1)]
na_cols = out.columns[out.isna().any()].tolist()
out[["year","iso"] + na_cols]

,year,iso,bill_rate,def_gdp,debtgdp,mil_gdp
231,1950,BEL,1.40,-5.179133,73.688900,NaN
232,1951,BEL,1.40,-3.272222,64.479400,NaN
233,1952,BEL,1.40,-5.771846,66.252200,NaN
261,1980,BEL,14.08,-9.097519,NaN,3.250195
262,1981,BEL,15.25,-14.285372,NaN,3.366049
...,...,...,...,...,...,...
2501,1955,SWE,3.69,-1.545027,27.804388,NaN
2502,1956,SWE,4.03,-1.083404,27.150103,NaN
2503,1957,SWE,4.60,-1.888203,27.541811,NaN
2504,1958,SWE,4.67,-2.055053,28.621067,NaN


In [13]:
df.iloc[:, 2:]=df.iloc[:, 2:].interpolate()
df.iloc[:, 2:]=df.iloc[:, 2:].bfill()

df = df.loc[df.year >= 1950]

df = df.copy()

In [14]:
vars_keep = ["rgdp_pc","gov_pc","bill_rate","def_gdp","debtgdp","tfp","mil_gdp","nx_gdp"]

# logs (only where strictly positive)
df["log_rgdp_pc"] = np.log(df["rgdp_pc"])
df["log_gov_pc"]  = np.log(df["gov_pc"])
df["log_tfp"]     = np.log(df["tfp"])

# ratios/rates typically already in percent points (just keep)
df["tbill"]    = df["bill_rate"]
df["def_gdp"]  = df["def_gdp"]
df["debt_gdp"] = df["debtgdp"]
df["mil_gdp"]  = df["mil_gdp"]
df["nx_gdp"]   = df["nx_gdp"]

model_vars = ["log_rgdp_pc","log_gov_pc","log_tfp","tbill","def_gdp","debt_gdp","mil_gdp","nx_gdp"]

In [15]:
df = df.sort_values(["iso","year"])

# create full grid iso x year
isos  = sorted(df["iso"].unique())
years = np.arange(df["year"].min(), df["year"].max()+1)

grid = pd.MultiIndex.from_product([isos, years], names=["iso","year"]).to_frame(index=False)
dfg = grid.merge(df[["iso","year"] + model_vars], on=["iso","year"], how="left")

# find years that are complete for ALL countries and ALL vars
complete_by_year = (dfg.groupby("year")[model_vars]
                      .apply(lambda g: g.notna().all().all()))  # True if no NaNs anywhere that year

years_ok = complete_by_year[complete_by_year].index
df = dfg[dfg["year"].isin(years_ok)].copy()

# final check: no missing
assert df[model_vars].notna().all().all()

In [16]:
df

,iso,year,log_rgdp_pc,log_gov_pc,log_tfp,tbill,def_gdp,debt_gdp,mil_gdp,nx_gdp
0,BEL,1950,1.874636,0.444990,1.354683,1.4000,-5.179133,73.6889,4.862558,-4.414455
1,BEL,1951,1.937201,0.456351,1.403621,1.4000,-3.272222,64.4794,4.862558,1.317299
2,BEL,1952,1.940369,0.556753,1.400661,1.4000,-5.771846,66.2522,4.862558,-0.099480
3,BEL,1953,1.949283,0.487863,1.440028,1.4000,-4.360491,68.6405,4.862558,-1.987730
4,BEL,1954,1.971972,0.490545,1.477510,1.4000,-4.873439,69.6069,4.880254,-2.875324
...,...,...,...,...,...,...,...,...,...,...
918,SWE,2016,3.953127,2.735742,2.548431,-0.6550,1.465335,42.2554,1.055958,-0.222739
919,SWE,2017,3.969385,2.743045,2.555912,-0.6950,1.650562,40.7310,1.032598,-0.175045
920,SWE,2018,3.982214,2.762205,2.557299,-0.6858,1.359689,38.9087,1.040003,-0.465028
921,SWE,2019,3.999731,2.764608,2.572266,-0.4208,1.337586,34.8980,1.096923,0.419596


In [17]:
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df = df.sort_values(["iso", "year"])
df = df.set_index(["iso", "year"])

In [18]:
df

log_rgdp_pc  log_gov_pc   log_tfp   tbill   def_gdp  debt_gdp  \
iso year                                                                  
BEL 1950     1.874636    0.444990  1.354683  1.4000 -5.179133   73.6889   
    1951     1.937201    0.456351  1.403621  1.4000 -3.272222   64.4794   
    1952     1.940369    0.556753  1.400661  1.4000 -5.771846   66.2522   
    1953     1.949283    0.487863  1.440028  1.4000 -4.360491   68.6405   
    1954     1.971972    0.490545  1.477510  1.4000 -4.873439   69.6069   
...               ...         ...       ...     ...       ...       ...   
SWE 2016     3.953127    2.735742  2.548431 -0.6550  1.465335   42.2554   
    2017     3.969385    2.743045  2.555912 -0.6950  1.650562   40.7310   
    2018     3.982214    2.762205  2.557299 -0.6858  1.359689   38.9087   
    2019     3.999731    2.764608  2.572266 -0.4208  1.337586   34.8980   
    2020     3.976694    2.870453  2.566931 -0.1350 -2.703437   39.6647   

           mil_gdp    nx_gdp  
iso year                      
BEL 1950  4.862558 -4.414455  
    1951  4.862558  1.317299  
    1952  4.862558 -0.099480  
    1953  4.862558 -1.987730  
    1954  4.880254 -2.875324  
...            ...       ...  
SWE 2016  1.055958 -0.222739  
    2017  1.032598 -0.175045  
    2018  1.040003 -0.465028  
    2019  1.096923  0.419596  
    2020  1.150095  0.994851  

[923 rows x 8 columns]

In [19]:
g = df["mil_gdp"].groupby(level=0)
df["mil_delta"] = g.shift(-1) - df["mil_gdp"]

In [20]:
df = df.drop(columns=["mil_gdp"])
df = df.dropna()

In [21]:
df

log_rgdp_pc  log_gov_pc   log_tfp     tbill   def_gdp  debt_gdp  \
iso year                                                                    
BEL 1950     1.874636    0.444990  1.354683  1.400000 -5.179133   73.6889   
    1951     1.937201    0.456351  1.403621  1.400000 -3.272222   64.4794   
    1952     1.940369    0.556753  1.400661  1.400000 -5.771846   66.2522   
    1953     1.949283    0.487863  1.440028  1.400000 -4.360491   68.6405   
    1954     1.971972    0.490545  1.477510  1.400000 -4.873439   69.6069   
...               ...         ...       ...       ...       ...       ...   
SWE 2015     3.941785    2.733937  2.552606 -0.289167  0.149843   43.7387   
    2016     3.953127    2.735742  2.548431 -0.655000  1.465335   42.2554   
    2017     3.969385    2.743045  2.555912 -0.695000  1.650562   40.7310   
    2018     3.982214    2.762205  2.557299 -0.685800  1.359689   38.9087   
    2019     3.999731    2.764608  2.572266 -0.420800  1.337586   34.8980   

            nx_gdp  mil_delta  
iso year                       
BEL 1950 -4.414455   0.000000  
    1951  1.317299   0.000000  
    1952 -0.099480   0.000000  
    1953 -1.987730   0.017696  
    1954 -2.875324  -1.096842  
...            ...        ...  
SWE 2015  0.416597  -0.017783  
    2016 -0.222739  -0.023360  
    2017 -0.175045   0.007405  
    2018 -0.465028   0.056920  
    2019  0.419596   0.053172  

[910 rows x 8 columns]

In [22]:
df = df.reset_index()

In [23]:
df.to_csv(Path.cwd().parent / "data processed" / "nato_dataset.csv", index=False)

In [24]:
df.iso.unique()

array(['BEL', 'CHE', 'DNK', 'ESP', 'FIN', 'FRA', 'GBR', 'IRL', 'ITA',
       'NLD', 'NOR', 'PRT', 'SWE'], dtype=object)